In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [6]:
data_dir='D:\\Msc GU\\Msc 4th Sem\\Project\\DataSet\\teadataset'

In [7]:
# Set up data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

# Load in the dataset
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 711 images belonging to 8 classes.
Found 174 images belonging to 8 classes.


In [8]:
# Load in the pre-trained NASNetMobile model
base_model = tf.keras.applications.NASNetMobile(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet')

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [9]:
 #Add some layers on top of the pre-trained model
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(8, activation='softmax')(x)

# Create the final model
model = tf.keras.models.Model(inputs=base_model.input, outputs=x)

In [12]:
 #Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using transfer learning
history = model.fit(train_data, epochs=50, validation_data=val_data)

Epoch 1/50
23/23 [==============================] - 513s 16s/step - loss: 0.9213 - accuracy: 0.6554 - val_loss: 1.8594 - val_accuracy: 0.5287
Epoch 2/50
23/23 [==============================] - 323s 14s/step - loss: 0.6182 - accuracy: 0.7637 - val_loss: 1.6723 - val_accuracy: 0.5517
Epoch 3/50
23/23 [==============================] - 319s 14s/step - loss: 0.4078 - accuracy: 0.8439 - val_loss: 1.7058 - val_accuracy: 0.5575
Epoch 4/50
23/23 [==============================] - 281s 12s/step - loss: 0.3145 - accuracy: 0.8734 - val_loss: 1.6090 - val_accuracy: 0.5805
Epoch 5/50
23/23 [==============================] - 285s 12s/step - loss: 0.3617 - accuracy: 0.8664 - val_loss: 1.9744 - val_accuracy: 0.5690
Epoch 6/50
23/23 [==============================] - 281s 12s/step - loss: 0.3501 - accuracy: 0.8692 - val_loss: 1.3917 - val_accuracy: 0.6207
Epoch 7/50
23/23 [==============================] - 290s 13s/step - loss: 0.2440 - accuracy: 0.9086 - val_loss: 2.1405 - val_accuracy: 0.6149
Epoch 

In [13]:
 #Unfreeze the top layers of the base model
for layer in base_model.layers[-120:]:
    layer.trainable = True

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history = model.fit(train_data, epochs=50, validation_data=val_data)


Epoch 1/50
23/23 [==============================] - 186s 6s/step - loss: 0.1642 - accuracy: 0.9494 - val_loss: 24.2570 - val_accuracy: 0.2989
Epoch 2/50
23/23 [==============================] - 126s 6s/step - loss: 0.1360 - accuracy: 0.9466 - val_loss: 19.2983 - val_accuracy: 0.4425
Epoch 3/50
23/23 [==============================] - 124s 5s/step - loss: 0.1250 - accuracy: 0.9508 - val_loss: 16.6101 - val_accuracy: 0.4598
Epoch 4/50
23/23 [==============================] - 154s 7s/step - loss: 0.1529 - accuracy: 0.9662 - val_loss: 12.5395 - val_accuracy: 0.4885
Epoch 5/50
23/23 [==============================] - 131s 6s/step - loss: 0.1337 - accuracy: 0.9620 - val_loss: 9.1839 - val_accuracy: 0.6034
Epoch 6/50
23/23 [==============================] - 134s 6s/step - loss: 0.1692 - accuracy: 0.9508 - val_loss: 7.7214 - val_accuracy: 0.6092
Epoch 7/50
23/23 [==============================] - 140s 6s/step - loss: 0.1368 - accuracy: 0.9606 - val_loss: 7.1857 - val_accuracy: 0.6207
Epoch 8/5

In [16]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3)

test_data = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size= 32,
    shuffle=True,
    class_mode='categorical',
    subset='validation')

Found 262 images belonging to 8 classes.


In [17]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_data)

# Print the results
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

9/9 [==============================] - 61s 7s/step - loss: 20.1335 - accuracy: 0.4656
Test Loss: 20.133527755737305
Test Accuracy: 0.4656488597393036


In [12]:
model.save('D:\\Msc GU\\Msc 4th Sem\\Project\\Complete Project\\FintuneNasNetonTeaDt.h5')